# Scoring public housing (HDB) data: calculation of number of amenities in the vicinity

Ansel Lim

4 November 2021, updated Dec 2021

HDB flats are public housing apartments in Singapore. Given HDB dataset combined with latitude & longitude information, as well as coordinate data for various amenities / places of interest (taxi stands, schools, mrt, hawker centers, etc.), calculate counts of amenities within a 1km radius *for each HDB block*.

In [1]:
# Specify radius in kilometers
RADIUS = 1.0

In [2]:
import os
import time
from datetime import datetime
import pandas as pd
from geopy.distance import geodesic

In [3]:
timestamp = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
print(timestamp)  # GMT

04/12/2021 17:06:53


Load raw datasets

In [4]:
os.chdir("../data/raw/")

In [5]:
# Use `hdb_aggregated.csv` (that is, apartments in the same block are considered as belonging to the same project, so we aggregate data on a block-by-block basis rather than looking at every single apartment)

# Amenities
hdb = pd.read_csv("./hdb_aggregated.csv")
malls = pd.read_csv("./data_malls.csv")
taxi_stands = pd.read_csv("./taxi_stands.csv")
primary_schools = pd.read_csv("./data_prischools.csv")
mrt = pd.read_csv("./data_MRT.csv")
hawker = pd.read_csv("./data_hawker.csv")
carparks = pd.read_csv("./carparks.csv")
bus_stops = pd.read_csv("./bus_stops.csv")
amenities = pd.read_csv("./amenities.csv")
supermarkets = pd.read_csv("./supermarkets.csv")
secondary_schools = pd.read_csv("./secondary_schools.csv")
eating_establishments = pd.read_csv("./eating_establishments.csv")
parks = pd.read_csv("./parks.csv")

In [6]:
print("Number of HDB blocks in dataset",hdb.shape[0])

Number of HDB blocks in dataset 9148


Only use the HDB blocks with latitude and longitude information available

In [7]:
hdb = hdb[(hdb["isLatLongAvailable"] == True)]
hdb.reset_index(inplace=True)

In [8]:
print("Number of HDB blocks in dataset with latitude & longitude data",hdb.shape[0])

Number of HDB blocks in dataset with latitude & longitude data 9126


The datasets are from different sources. Some datasets store the latitude and longitude data in a "Coordinates" column, so we need to harmonize the column names.

In [9]:
taxi_stands["lat"], taxi_stands["long"] = taxi_stands["Latitude"], taxi_stands["Longitude"]
primary_schools[['long', 'lat']] = primary_schools['coordinates'].str.split(',', 1, expand=True)
mrt[['long', 'lat']] = mrt['Coordinates'].str.split(',', 1, expand=True)
hawker[['long', 'lat']] = hawker['Coordinates'].str.split(',', 1, expand=True)
hawker['lat'] = hawker['lat'].str.rstrip(",0.0")
carparks['lat'], carparks['long'] = carparks['latitude'], carparks['longitude']
bus_stops['lat'], bus_stops['long'] = bus_stops['Latitude'], bus_stops['Longitude']

The `amenities.csv` file contains information about CHAS clinics (subsidized healthcare facilities), sports facilities (gyms, swimming pools, etc.), and community centers.

In [10]:
sports_facility_types = list(amenities.facility_type.unique())
sports_facility_types.remove('CHAS Clinic')
sports_facility_types.remove('Community Centre')
chas_clinics = amenities[amenities['facility_type'] == 'CHAS Clinic'].reset_index()
sports_facilities = amenities[amenities['facility_type'].isin(sports_facility_types)].reset_index()
community_centers = amenities[amenities['facility_type'] == 'Community Centre'].reset_index()

In [11]:
places = [malls, taxi_stands, primary_schools, mrt, hawker, carparks, bus_stops, chas_clinics, sports_facilities,
          community_centers, supermarkets, secondary_schools, eating_establishments, parks]
places_names = ['malls', 'taxi_stands', 'primary_schools', 'mrt', 'hawker', 'carparks', 'bus_stops', 'chas_clinics',
                'sports_facilities', 'community_centers', 'supermarkets', 'secondary_schools', 'eating_establishments',
                'parks']

In [12]:
df = hdb.copy(deep=True)

In [ ]:
start = time.time()
interm = time.time()
for i in range(len(places)):
    dataframe = places[i]
    dataframe.reset_index(inplace=True)
    place_name = places_names[i]
    print("-------------------------")
    print("Working on feature dataframe {}".format(place_name))
    print("Number of places of interest:", dataframe.shape[0])
    print("Estimated number of pairwise computations:", dataframe.shape[0] * df.shape[0])
    new_column_name = "num_" + place_name
    df[new_column_name] = 0
    for i in range(df.shape[0]):
        lat1 = df.loc[i, "lat"]
        long1 = df.loc[i, "long"]
        origin = (lat1, long1)
        counter = 0
        for j in range(dataframe.shape[0]):
            lat2 = dataframe.loc[j, "lat"]
            long2 = dataframe.loc[j, "long"]
            dest = (lat2, long2)
            try:
                dist = geodesic(origin, dest).km
                if dist <= RADIUS:
                    counter += 1
            except ValueError:
                continue
        df.loc[i, new_column_name] = counter
    df.to_csv('../processed/df_hdb.csv')  # save checkpoint
    print("Saved checkpoint")
    print("Completed working on feature dataframe {}".format(place_name))
    print("Time taken for this feature dataframe (seconds):", time.time() - interm)
    interm = time.time()
    print("Total time elapsed since start (seconds):", (time.time() - start))
end = time.time()
print("---------------------------------------")
print("END OF ALL TASKS")
print("time taken (seconds): {}".format(end - start))

-------------------------
Working on feature dataframe malls
Number of places of interest: 169
Estimated number of pairwise computations: 1542294
Saved checkpoint
Completed working on feature dataframe malls
Time taken for this feature dataframe (seconds): 263.35767793655396
Total time elapsed since start (seconds): 263.3577148914337
-------------------------
Working on feature dataframe taxi_stands
Number of places of interest: 279
Estimated number of pairwise computations: 2546154
Saved checkpoint
Completed working on feature dataframe taxi_stands
Time taken for this feature dataframe (seconds): 438.12201714515686
Total time elapsed since start (seconds): 701.4797530174255
-------------------------
Working on feature dataframe primary_schools
Number of places of interest: 186
Estimated number of pairwise computations: 1697436
Saved checkpoint
Completed working on feature dataframe primary_schools
Time taken for this feature dataframe (seconds): 296.4494049549103
Total time elapsed si

In [ ]:
df.to_csv('../processed/df_hdb.csv')

In [ ]:
1+1